In [1]:
import statsmodels.api as sm
import numpy as np
import pandas_datareader as web
import yfinance as yf
import pandas as pd
ff_data = pd.read_csv("https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip", 
                      skiprows=3)
ff_data.rename(columns={0:'Date'}, inplace=True)
ff_data.columns = ff_data.columns.str.replace('Unnamed: 0','Date')
ff_data = ff_data[~ff_data.astype(str).apply(lambda x: x.str.contains('Annual Factors: January-December')).any(axis=1)]
ff_data = ff_data[ff_data['Date'].str.strip().str.len() == 6]
max_date = ff_data['Date'].max()
ff_data.set_index('Date', inplace=True)
ff_data = ff_data.loc[:"max_date"]
ff_data.index = pd.to_datetime(ff_data.index, format="%Y%m").to_period('M')
ff_data = ff_data.apply(pd.to_numeric, errors='coerce')
ff_data = ff_data/100
ff_data

Mkt-RF     SMB     HML      RF
Date                                   
1926-07  0.0296 -0.0256 -0.0243  0.0022
1926-08  0.0264 -0.0117  0.0382  0.0025
1926-09  0.0036 -0.0140  0.0013  0.0023
1926-10 -0.0324 -0.0009  0.0070  0.0032
1926-11  0.0253 -0.0010 -0.0051  0.0031
...         ...     ...     ...     ...
2023-01  0.0665  0.0502 -0.0405  0.0035
2023-02 -0.0258  0.0121 -0.0078  0.0034
2023-03  0.0251 -0.0559 -0.0901  0.0036
2023-04  0.0061 -0.0334 -0.0003  0.0035
2023-05  0.0035  0.0153 -0.0780  0.0036

[1163 rows x 4 columns]

In [2]:
df = yf.download('NSBRX', 
                 start='2007-04-01', 
                 end='2021-12-31',
                 progress=False)
prices = df.loc[:, ['Adj Close']]
returns_d = prices/prices.shift(1) - 1
returns_m = returns_d.resample('M').apply(lambda x: (x + 1).prod() - 1).to_period('M')

In [3]:
eg_excess = returns_m["2007-04":"2021-12"] - ff_data.loc["2007-04":"2021-12", ['RF']].values
mkt_excess = ff_data.loc["2007-04":"2021-12",['Mkt-RF']]
exp_var = mkt_excess.copy()
exp_var["Constant"] = 1
exp_var["Value"] = ff_data.loc["2007-04":"2021-12",['HML']]
exp_var["Size"] = ff_data.loc["2007-04":"2021-12",['SMB']]

In [4]:
lm = sm.OLS(eg_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     770.5
Date:                Sun, 23 Jul 2023   Prob (F-statistic):          8.10e-100
Time:                        14:56:55   Log-Likelihood:                 552.31
No. Observations:                 177   AIC:                            -1097.
Df Residuals:                     173   BIC:                            -1084.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.8681      0.019     45.117      0.000       0.830       0.906
Constant       0.0008      0.001      0.949      0.344      -0.001       0.002
Value          0.0581      0.027      2.123      0.035       0.004       0.112
Size          -0.1954      0.036     -5.460      0.000      -0.266      -0.125
==============================================================================
Omnibus:                       30.694   Durbin-Watson:                   2.092
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               54.944
Skew:                           0.858   Prob(JB):                     1.17e-12
Kurtosis:                       5.123   Cond. No.                         45.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""